## Note : 
Mean free path : $\lambda = \frac{1}{n \sigma}$

If $n \rightarrow 100 \times n$, then :
- $\lambda \rightarrow \frac{\lambda}{100}$
- $\text{cell size} \rightarrow \frac{\text{cell size}}{100}$ ($100^2$ more cells because the grid is 2D)
- $dt \rightarrow \frac{dt}{100}$

But you can lower, down to a certain extent, the number of particles per cell to "not incease too much the computation time".

## Representative physical length scale

The system has two lenghts : 
* lenght of the tube : $L = 0.01$ $m$
* width : $w = 0.001$ $m$

The width is the representative physical length scale that constrains the flow.

So : $K_d = \frac{\lambda}{w}$.




In [2]:
%matplotlib widget

# notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as scipy
from pathlib import Path

from lppydsmc.data.saver import Saver
from plotting import analysis
from lppydsmc.utils import physics

plt.figure.max_open_warning = False

## Loading results

In [3]:
dir_path = Path('results/')

simu = 'low' # low, mid, thruster_low, test_hall

dz = 0.001

In [4]:
if(simu == 'low'):
    dt = 1e-6
    mr = 32000000
    name = 'test_hard_reset.h5'
    lenght = 0.01 # m
    width = 0.001
    nb_cells = 10
    cell_size = 0.001
    temperature = 300 # K
    volume = cell_size*lenght*dz
elif(simu == 'test_tube_low_no_inject'):
    dt = 1e-6 
    mr = 3.200000e+07
    name = 'test_tube_low_no_inject.h5'
    lenght = 0.01 # m
    width = 0.001
    nb_cells = 10
    cell_size = 0.001
    temperature = 300 # K
    volume = cell_size*lenght*dz
elif(simu == 'low_with_init'):
    dt = 1e-6 
    mr = 3.200000e+07
    name = 'test_tube_low_with_init.h5'
    lenght = 0.01 # m
    width = 0.001
    nb_cells = 10
    cell_size = 0.001
    temperature = 300 # K
    volume = cell_size*lenght*dz
elif(simu == 'low_wave'):
    dt = 1e-7 
    mr = 3.200000e+06
    name = 'test_tube_low_wave.h5'
    lenght = 0.01 # m
    width = 0.001
    nb_cells = 10
    cell_size = 0.001
    temperature = 300 # K
    volume = cell_size*lenght*dz
elif(simu == 'mid'):
    dt = 5e-07
    mr = 1.6e6
    name = 'test_tube_mid.h5'
    lenght = 0.01
    width = 0.001
    cell_size = 0.0001
    nb_cells = 100
    temperature = 300 # K
    volume = cell_size*lenght*dz
elif(simu == 'high'):
    dt = 5e-9 
    mr = 3.200000e+11
    name = 'test_tube_high.h5'
    lenght = 0.01
    width = 0.001
    cell_size = 0.000001 # not feasible
    temperature = 300 # K
    volume = cell_size*lenght*dz
elif(simu == 'big'):
    dt = 1e-6 
    mr = 3.200000e+07
    name = 'test_big_tube.h5'
    lenght = 0.1
    width = 0.001
    cell_size = 0.001
    temperature = 300 # K
    volume = cell_size*lenght*dz
elif(simu == 'thruster_low'): # representative lenght scale varies along the flow ...
    dt = 1e-6 
    mr = 3.200000e+07
    name = 'test_thruster_low.h5'
    lenght = 0.011 # m
    width = 0.005
    cell_size = 0.001
    temperature = 300 # K
    nb_cells = 33
    volume = cell_size*cell_size*dz*nb_cells
elif(simu == 'test_hall'): # representative lenght scale varies along the flow ...
    dt = 1e-5
    mr = 160000000
    name = 'test_hall.h5'
    lenght = 0.025 # m
    width = 0.01
    cell_size = 0.001
    temperature = 640 # K
    nb_cells = 20*25
    volume = cell_size*cell_size*dz*nb_cells

In [5]:
store = pd.HDFStore(dir_path/name)

In [6]:
# defining each variable
collisions_per_cell_full = store['collisions_per_cell'] 
df_full = store['df']
pmax_per_cell = store['pmax_per_cell'] # cell-discretization not taken into account 
averages_per_cell = store['averages_per_cell'] # idem
total_deleted = store['total_deleted']
total_distance = store['total_distance']
total_proba = store['total_proba']
collisions_with_walls = store['collisions_with_walls']

In [7]:
choice = 'I'
gamma = 5/3. # roughly for atomes, for diatomic molecule : 7/5.

if(choice=='I'):
    mass = physics.get_mass_part(53, 53, 74) # I
    molecular_mass = 0.1269 # kg/mol
elif(choice=='Xe'):
    mass = 131.293*physics.ATOMIC_MASS # Xe
    molecular_mass = 0.131293
    
verbose=False

In [8]:
sound_vel = physics.speed_of_sound(molecular_mass, temperature, gamma)
print(f'Speed of sound for [{choice}] at temperature {temperature} K : {sound_vel} m/s')

Speed of sound for [I] at temperature 300 K : 180.99192585903475 m/s


## Note : 
To save a figure, you can use : `analysis.save_fig(fig, path, title = None, dpi = 400, figsize = None)`, which allows you to easily save a figure while giving it a title and changing its size.
You can also use the interactive widget.

## Choosing the part of the system to plot 

In [9]:
# grid resolutions for the thruster

thruster = False

if(thruster):
    res_x_0, res_y_0 = 11, 5
else:
    res_x_0, res_y_0 = 10, 1 # 100, 10 # 25, 10

total_nb_cells = res_x_0*res_y_0

In [10]:
fig, ax = plt.subplots(1, constrained_layout = True, figsize = (res_x_0, res_y_0));

fact = 10
volume_cell_plot = volume/(fact*fact*res_x_0*res_y_0)
duration = df_full.index.unique().values.shape[0]
analysis.hist2d(ax, df_full, bins = (fact*res_x_0,fact*res_y_0), weights = mr*np.ones(df_full.shape[0])/(duration*volume_cell_plot), stat = 'count'); # TODO: add cmap

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
use_part = False

# tube last part
if(use_part):
    if(thruster):
        part = 1 # 2
        if(part == 1):
            x_min, x_max = 0, 3e-3 
            y_min, y_max = 0.0, 5e-3
            
            # collisions
            res_x, res_y = 3, 5 
            offset = 0
            cells = [k for k in range(15)]
            
        elif(part == 2):
            x_min, x_max = 0.005, 1e-2
            y_min, y_max = 2e-3, 3e-3
            
            # collisions
            res_x, res_y = 5,1
            offset = 27
            cells = [27, 32, 37, 42, 47] 
            
        volume = (x_max-x_min)*(y_max-y_min)*dz
        df = df_full.loc[(df_full['x']>x_min) & (df_full['x']<x_max) & (df_full['y']>y_min) & (df_full['y']<y_max)].copy(deep=True)

        # in addition we want to select the right cells in collisions
        # plotting the mfp for the selected cells.
        # first along x axis, then along y axis
        nb_rows = collisions_per_cell_full.shape[0]
        
        index_collisions = collisions_per_cell_full.index.unique().values
        collisions_per_cell = np.zeros((res_x,res_y,index_collisions.shape[0]))

        for cell in cells :
            i, j = (cell-offset)//res_y_0, (cell-offset)%res_y_0
            collisions_per_cell[i,j,:] = collisions_per_cell_full[cell:nb_rows:total_nb_cells].values
    else: # tube
        hall = False
        if(not hall):
            x_min, x_max = 9e-3, 1e-2
            y_min, y_max = 0, 1e-3

            res_x, res_y = 1, 1
            offset = 9 
            cells = [9]
        else:
            x_min, x_max = 2e-2, 2.5e-2
            y_min, y_max = 0, 1e-2

            res_x, res_y = 5, 10 
            offset = 200 
            cells = [offset+k for k in range(50)] 
            
        volume = (x_max-x_min)*(y_max-y_min)*dz
        df = df_full.loc[(df_full['x']>x_min) & (df_full['x']<x_max) & (df_full['y']>y_min) & (df_full['y']<y_max)].copy(deep=True)
        # in addition we want to select the right cells in collisions
        # plotting the mfp for the selected cells.
        # first along x axis, then along y axis
            
        nb_rows = collisions_per_cell_full.shape[0]
        
        index_collisions = collisions_per_cell_full.index.unique().values
        collisions_per_cell = np.zeros((res_x,res_y,index_collisions.shape[0]))

        for cell in cells :
            i, j = (cell-offset)//res_y_0, (cell-offset)%res_y_0
            collisions_per_cell[i,j,:] = collisions_per_cell_full[cell:nb_rows:total_nb_cells].values
        
else:
    df = df_full # another reference.
    nb_rows = collisions_per_cell_full.shape[0]

    ## we are still
    index_collisions = collisions_per_cell_full.index.unique().values
    collisions_per_cell = np.zeros((res_x_0,res_y_0,index_collisions.shape[0]))
    for cell in range(total_nb_cells):
        i, j = cell//res_y_0, cell%res_y_0
        collisions_per_cell[i,j,:] = collisions_per_cell_full[cell:nb_rows:total_nb_cells].values

## Choosing frames to plot

In [12]:
unique_index = df.index.unique().values
nb_save = unique_index.shape[0]
iterations = np.max(unique_index)
adding_period = unique_index[1]-unique_index[0] # adding period - required to
# generally speaking, you choose frames so you have the steady state
frames = unique_index[int(0.8*nb_save):nb_save] 

if(verbose):
    print(f'Available frames :  {unique_index}')
    print(f'Max iteration : {iterations}')
    print(f'Choosen frames (for plotting) : \n{frames}')

## Particles analysis
### Plotting number of particles evolution

In [13]:
nb_parts = df['x'].groupby(df.index).agg('count').values

In [14]:
fig, ax = plt.subplots()
ax.plot(df.index.unique()*dt, nb_parts)
analysis.set_axis(ax, x = 'time', y = 'quantity')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Plotting density evolution

In [15]:
fig, ax = plt.subplots()
ax.plot(df.index.unique()*dt, nb_parts*mr/volume)
analysis.set_axis(ax, x = 'time', y = 'density')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Density as a function of x

In [16]:
fig, ax = plt.subplots()
bins = 100
volume_cell_plot = volume/bins
duration = df.index.unique().values.shape[0]
weights = mr*np.ones(df.shape[0])/(duration*volume_cell_plot)
analysis.hist1d(ax, df, 'x', bins = bins, density = False, color = 'default', weights = weights, histtype = 'step')
analysis.set_axis(ax, x = 'x', y = 'density');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Pressure in mTorr

In [16]:
pressure_torr = physics.pressure_torr(nb_parts*mr, volume, temperature)

In [17]:
fig, ax = plt.subplots()
ax.plot(df.index.unique()*dt, pressure_torr*1e3)
analysis.set_axis(ax, x = 'time', y = None)
ax.set_ylabel('pressure (mTorr)');

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Final state

In [18]:
fig, ax = plt.subplots()
analysis.state(ax, df.loc[df.index == frames[-1]], c = None)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Velocity distribution

In [19]:
fig, axes = analysis.velocity_distribution(df, frames, bins = 100, density = True, sharex = False, sharey = False);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

df_mean = df.groupby(df.index).mean()
## Temperature evolution

First - not by cell but for the whole system.

### Theory : 

The kinetic energy density $w$ is

$w = \frac{3}{2}p(\vec{r},t) + n(\vec{r},t)\frac{1}{2} m |\vec{u}(\vec{r},t)|^2$ 

With $\vec{u}(\vec{r},t) = ((<(v_x, v_y, v_z)>)(\vec{r},t))^T$ thus : $|\vec{u}(\vec{r},t)| = |<\vec{v}>|$.

Here the drift is not zero. So we have to remove it to have only $\frac{3}{2}p(\vec{r},t) = w - n(\vec{r},t)\frac{1}{2} m |\vec{u}(\vec{r},t)|^2$.

This yields : 

$\frac{3}{2}p(\vec{r},t) = n(\vec{r},t)  <\frac{1}{2}mv^2> - n(\vec{r},t)\frac{1}{2} m |\vec{u}(\vec{r},t)|^2$

Which yields : 

$$ \frac{3}{2}p(\vec{r},t) = \frac{m n(\vec{r},t)}{2}  (<v^2> - |<\vec{v}>|^2) $$

With $p = nkT$.

Intuition : total energy - energy contains in the drift.

In [20]:
df['v'] = np.sqrt(df['vx']*df['vx']+df['vy']*df['vy']+df['vz']*df['vz']) # df.apply(analysis.speed_norm, axis = 1)
df['v2'] = df['v']**2
df_mean = df.groupby(df.index).mean()
df_mean['v_drift'] = np.sqrt(df_mean['vx']*df_mean['vx']+df_mean['vy']*df_mean['vy']+df_mean['vz']*df_mean['vz'])

In [21]:
# maxwellian temperature 
temperature_mb = np.pi*mass*df_mean['v']**2/(8*physics.BOLTZMAN_CONSTANT)

In [22]:
df_mean['temperature'] =  mass/(3.*physics.BOLTZMAN_CONSTANT) * (df_mean['v2']-df_mean['v_drift']**2)
temperatures = df_mean['temperature']

In [23]:
fig, ax = plt.subplots(1,2)
ax[0].plot(unique_index*dt,temperatures)
ax[1].plot(unique_index*dt,temperature_mb)

analysis.set_axis(ax[0], x = 'time', y = 'temperature')
analysis.set_axis(ax[1], x = 'time', y = 'temperature')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
fig, ax = plt.subplots()
ax.plot(unique_index*dt,temperatures)
analysis.set_axis(ax, x = 'time', y = 'temperature')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
fig, ax = plt.subplots()
ax.plot(unique_index*dt, physics.pressure_torr(nb_parts*mr, volume, temperature_mb)*1e3)
analysis.set_axis(ax, x = 'iteration', y = None)
ax.set_ylabel('pressure (mTorr)');

In [ ]:
sound_speed = physics.speed_of_sound(molecular_mass, temperature_mb, gamma)

In [ ]:
fig, ax = plt.subplots()
ax.plot(unique_index*dt,sound_speed)
analysis.set_axis(ax, x = 'time', y = 'sound speed')

## Flow speed

In [ ]:
fig, ax = plt.subplots()
ax.plot(unique_index*dt,df_mean['v_drift'])
analysis.set_axis(ax, x = 'time', y = 'v');

## Mach number

In [ ]:
fig, ax = plt.subplots()
ax.plot(unique_index*dt,df_mean['v_drift']/sound_speed)
analysis.set_axis(ax, x = 'time')
ax.set_ylabel('Mach number');

## Collision with walls - analysis

In [ ]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'quantity')
ax.plot(collisions_with_walls.index*dt, collisions_with_walls*mr);

## DSMC collisions analysis

### Particles collisions analysis

In [ ]:
# start by summing over all cells
collisions = np.sum(collisions_per_cell, axis = (0,1))
print('Total number of collision in the simulation (in reality) : {:e} ({:e})'.format(int(np.sum(collisions_per_cell)),mr*np.sum(collisions_per_cell)));

In [ ]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'quantity')
ax.plot(index_collisions, collisions*mr);

### Rate between the two

In [ ]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time')
ax.set_ylabel('rate')
ax.plot(index_collisions*dt, collisions_with_walls/collisions);

## Mean free path 
The formula for the mean free path is : $\lambda = \frac{v_{mean}}{\nu}$, where : 
* $v_{mean}$ is the mean velocity ;
* $\nu$ is the frequency of collision for ONE particle.

We have : $\frac{1}{\nu} = \frac{N_c dt}{2 N}$ with $N_c$ the number of particle in the cell, and $N$ the number of collisions.

This yields :  $\lambda = \frac{v_{mean}N_c dt}{2N}$. There is no need for $m_r$ here as both $N_c$ and $N$ would require the multiplication to have the current number in the system.

We start by doing it for the whole system.

In [ ]:
x_res = 10 # for example the number of cells during the simulations (if you want a plot per cell)
y_res = 1
x_step = 0.001 # size of the cell
y_step = 0.001

In [ ]:
v_mean = df_mean['v']

In [ ]:
# selecting the time iteration that we also saved for the collisions (saving frequency is not necessary the same)
v_mean = v_mean.loc[np.isin(v_mean.index,index_collisions)]
nb_particles_in_system = df['x'].groupby(df.index).agg('count').loc[np.isin(unique_index,index_collisions)]

In [ ]:
mfp = physics.mean_free_path_simu(nb_particles_in_system, collisions, dt, v_mean)

### Mean free path evolution

In [ ]:
fig, ax = plt.subplots()
ax.plot(index_collisions*dt, mfp)
analysis.set_axis(ax, x = 'time', y = 'mean free path')

### Knudsen evolution

In [ ]:
fig, ax = plt.subplots()
ax.plot(index_collisions*dt, mfp/cell_size)
analysis.set_axis(ax, x = 'time', y = 'Knudsen')

### Total mean free path

Accouting for collisions between particles and **collisions with walls**.

In the case where the number of collisions with walls is much bigger than the number of collisions between particles, then this mean free path is basically the average distance a particle covers between two wall collisions.

In [ ]:
mfp_ = 0.5*nb_particles_in_system*dt*v_mean/(collisions+collisions_with_walls)

#### Total Mean free path evolution 

In [ ]:
fig, ax = plt.subplots()
ax.plot(index_collisions*dt, mfp_)
analysis.set_axis(ax, x = 'time', y = 'mean free path');

#### Total Knudsen - evolution

In [ ]:
fig, ax = plt.subplots()
ax.plot(index_collisions*dt, mfp_/cell_size)
analysis.set_axis(ax, x = 'time', y = 'Knudsen');

## Out particle analysis

In [ ]:
fig, ax = plt.subplots()
mass_flow_rate = mr*physics.compute_mass_flow_rate(total_deleted, dt, mass)
analysis.set_axis(ax, x = 'time', y = 'mass flow rate')
plt.plot(mass_flow_rate.index*dt, mass_flow_rate);

### Mean number of particles

In [ ]:
averages = averages_per_cell.groupby(averages_per_cell.index).sum() # returns a pandas Series

In [ ]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'quantity')
ax.plot(averages.index*dt, averages);

### Mean acceptance probability

In [ ]:
proba = total_proba/collisions
print('Mean proba : {:e}'.format(np.mean(proba)))
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'probability')
ax.plot(proba.index*dt, proba, label = 'proba');

### Maximum probability

In [ ]:
pmax = pmax_per_cell.groupby(pmax_per_cell.index).sum() # returns a pandas Series

In [ ]:
fig, ax = plt.subplots()
analysis.set_axis(ax, x = 'time', y = 'probability')
ax.plot(pmax.index*dt, pmax);

### Mean distance between two colliding particles

In [ ]:
fig, ax = plt.subplots()
dist = total_distance/collisions
print('Mean distance : {:e} m'.format(np.mean(dist)))
analysis.set_axis(ax, x = 'time', y = 'distance')
ax.plot(dist.index*dt, dist);

### Velocity axis distribution per *cell*

In [ ]:
# spatial_hist2d plots an hist per cell defined by the params below
# it is a 2D plot of size y_res * x_res
# where each plot (i,j) takes the particle between i*x_step < x < (i+1)*x_step and j*y_step < y < (j+1)*y_step
# for i varying between between 0 and x_res-1, and j varying between between 0 and y_res-1
# You should not use too many cells because it quickly becomes very hard to view.
# Params
x_res = 11 # for example the number of cells during the simulations (if you want a plot per cell)
y_res = 5
x_step = 0.001 # size of the cell
y_step = 0.001 

# Plotting - vx
# fig, axes = analysis.spatial_hist2d(df, frames, val = 'vx', x_res = x_res, y_res = y_res, x_step = x_step, y_step=y_step);

In [ ]:
# Plotting - vy
# fig, axes = analysis.spatial_hist2d(df, frames, val = 'vy', x_res = x_res, y_res = y_res, x_step = x_step, y_step=y_step);

## Bonus - Not necessarily recommended and takes lots of time
Next is the plot of the system for each saved step. 

In [ ]:
# creating speed_norm (for the color)
# df['speed_norm'] = np.sqrt(df['vx']**2+df['vy']**2+df['vz']**2) -> DO NOT NEED THAT

In [ ]:
%matplotlib notebook
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation

def update_hist(num, df):
    dfit = df.loc[df.index == num]

    scat.set_offsets(np.c_[dfit['x'],dfit['y']])
    scat.set_array(df['v'])
    
    ax.set_title('{}/{}'.format(num+1, iterations), fontsize=15)

fig, ax = plt.subplots()
dfit = df.loc[df.index == 0]
scat = ax.scatter(dfit['x'], dfit['y'], s=0.1, c = dfit['v'], cmap='seismic') #  c = df['speed_norm']
analysis.set_axis(ax, x = 'x', y = 'y')
# if you want the boundaries to be plotted and the grid too, you have to initialize those fields yourself.
# in the other case, only particles will be plotted.
# plot_boundaries(ax, segments)
# plot_grid(ax, resolutions, system_shape)

ax.set_title('{}/{}'.format(1, iterations), fontsize=12)
ax.axis('equal')

interval = 40 # 25 images per second
anim = FuncAnimation(fig, update_hist, interval=interval, frames=iterations, fargs=(df, ), save_count=iterations)
anim.save(dir_path/'system_state_evo.avi', dpi = 300);